<a href="https://colab.research.google.com/github/Choi-SeoYun/2020DACONCUP/blob/main/%5BSY%5DDaconCup_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. library import

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 2. 데이터 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train = pd.read_csv("/content/gdrive/MyDrive/open_data/train.csv", encoding = 'euc-kr')
info_competition = pd.read_csv("/content/gdrive/MyDrive/open_data/info_competition.csv", encoding = 'euc-kr')
info_login = pd.read_csv("/content/gdrive/MyDrive/open_data/info_login.csv", encoding = 'euc-kr')
info_submission = pd.read_csv("/content/gdrive/MyDrive/open_data/info_submission.csv", encoding = 'euc-kr')
info_user = pd.read_csv("/content/gdrive/MyDrive/open_data/info_user.csv", encoding = 'euc-kr')
submission = pd.read_csv("/content/gdrive/MyDrive/open_data/submission.csv", encoding = 'euc-kr')

## 3. 데이터 살펴보기

### 1) info_competition

In [ ]:
info_competition.head()

,Unnamed: 0,cpt_id,is_official,period_start,period_end,name,name_eng,keyword,keyword_eng,participants,max_team_member,max_file_per_day,prize,prize_info,merge_deadline,winner_show
0,0,136,1,2018-08-14 0:00,2018-09-13 23:59,대출 상점 총 매출 예측 경진대회,Shop Total Sales Forecast,"금융 | 소상공인 신용카드 가맹점 빅데이터와 AI로 매출 예측 | 시계열, 회귀 ...",[Finance] Small Business Credit Card Merchant ...,303,5,5,850,"$8,500 + 100,000ZPR",2018-09-13 23:59,3
1,1,9565,1,2018-09-15 0:00,2018-10-13 23:59,병원 개/폐업 분류 예측 경진대회,hospital opening / closing classification pred...,금융 | 병원 재무 데이터와 AI로 개업|폐업 예측 분석 | 분류 | Accuracy,[Financial] Hospital financial data and openin...,448,10,3,350,"$3,500 + 40,000ZPR",2018-10-13 23:59,3
2,2,17801,1,2018-10-18 0:00,2018-12-31 23:59,아파트 경매가격 예측 경진대회,Apartment Auction Price Forecast,금융 | 부동산 아파트 경매 빅데이터와 AI로 경매가 예측 분석 | 회귀 | RMSE,[Finance] Real Estate Apartment Auction Big Da...,316,10,3,1000,"$10,000 + 120,000ZPR",2018-12-31 23:59,3
3,3,21265,1,2018-11-13 0:00,2019-01-31 23:59,아파트 실거래가 예측,apartment real price forecast,금융 | 부동산 빅데이터와 AI를 이용하여 실거래가를 예측 분석 | 회귀 | RMSE,[Finance] Predictive analysis of real transact...,568,10,3,800,"$8,000 + 80,000ZPR",2019-01-31 23:59,3
4,4,42473,1,2018-12-25 0:00,2019-01-10 23:59,신용카드 거래 데이터 시각화,credit card transaction data visualization,"금융 | 신용카드, 시계열, 시각화 | Python, R, Tableau, Spot...","[Finance] Credit cards, time series, visualiza...",252,5,3,100,"$1,000 + 100,000ZPR",2019-01-10 23:59,3


### 2) info_login

In [ ]:
info_login.head()

,Unnamed: 0,login_id,user_id,c_time,platform,browser
0,0.0,14196.0,19195.0,2018-09-23 1:30,Windows 8,Internet Explorer 10.0
1,1.0,14234.0,22045.0,2018-09-23 2:46,Windows,Chrome 67.0.3396.99
2,2.0,14256.0,8790.0,2018-09-23 3:35,Windows,Chrome 69.0.3497.100
3,3.0,14278.0,NaN,2018-09-23 12:50,Windows,Chrome 69.0.3497.100
4,4.0,14279.0,NaN,2018-09-23 12:50,Windows,Chrome 69.0.3497.100


### 3) info_submission

In [ ]:
info_submission.head()

,Unnamed: 0,sub_id,cpt_id,team_id,user_id,c_time
0,0.0,-250918.0,229255.0,5019.0,15880.0,2019-11-24 20:40
1,1.0,-250904.0,229611.0,5162.0,19772.0,2019-11-24 20:36
2,2.0,-250881.0,235401.0,5042.0,17801.0,2019-11-24 20:24
3,3.0,-250873.0,235401.0,5024.0,439.0,2019-11-24 20:23
4,4.0,-250860.0,229255.0,4890.0,13722.0,2019-11-24 20:19


### 4) info_user

In [ ]:
info_user.head()

,Unnamed: 0,id,c_time,entered_competition_cnt,code_share_cnt,talk_board_cnt,country_code,python_level,ds_level
0,0.0,15994.0,2017-05-02 23:58,94.0,306.0,477.0,82.0,1.0,1.0
1,1.0,19195.0,2018-08-08 18:21,1.0,0.0,0.0,82.0,1.0,1.0
2,2.0,16339.0,2018-08-08 19:57,1.0,0.0,0.0,82.0,1.0,1.0
3,3.0,7290.0,2018-08-08 19:59,0.0,0.0,0.0,82.0,1.0,1.0
4,4.0,21287.0,2018-08-08 20:26,0.0,0.0,0.0,82.0,1.0,1.0


### 5) submission

In [ ]:
submission.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,0,0,0,0
1,2020-11-10,0,0,0,0
2,2020-11-11,0,0,0,0
3,2020-11-12,0,0,0,0
4,2020-11-13,0,0,0,0


### 6) train

In [ ]:
train.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18


## 4. train data 불러오기

In [ ]:
train = pd.read_csv("/content/gdrive/MyDrive/open_data/train.csv", encoding = 'euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()

In [ ]:
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845
...,...,...,...,...,...
787,2020-11-04,4516,4472,1196,112683
788,2020-11-05,4155,4037,1044,102901
789,2020-11-06,3663,3576,825,88015
790,2020-11-07,2472,2417,531,57386


## 5. 데이터 가져와서 합치기(하려는 의도)
- 언니가 isHoliday를 넣은 것처럼 info_login정보를 넣어보자
- 먼저 info_login을 날짜별로 묶자(raw data는 시간별로 있음)
- 그 후, train과 info_login를 합치자
- **잠깐만!! 그럼 아이디 몇개가 평균적으로 몇번 들어왔는지 세야 하는건가? 합치는건 그냥 아이디값을 더하는 거니까! 위에처럼 하면 안됨!**

In [ ]:
info_login = pd.read_csv("/content/gdrive/MyDrive/open_data/info_login.csv", encoding = 'euc-kr')
info_login['c_time'] = pd.to_datetime(info_login.c_time)
info_login['date'] = info_login.c_time.dt.date
# 날짜별로 총 로그인 아이디 개수 세기, 중복되는 아이디 개수 세기
# 만약 9월23일의 로그인 아이디 대수가 161개 라면 소용 없어짐 -> 오케이 성공
login_size = info_login.groupby('date').size()
login_size = pd.DataFrame(login_size)
login_size = login_size.reset_index()
login_size.columns = ["date", "login_count"]

print(login_size.mean())
login_size

login_count    184.404703
dtype: float64


,date,login_count
0,2018-09-23,35
1,2018-09-24,30
2,2018-09-25,18
3,2018-09-26,48
4,2018-09-27,31
...,...,...
803,2020-12-04,276
804,2020-12-05,143
805,2020-12-06,169
806,2020-12-07,345


In [ ]:
my_train = pd.merge(train, login_size[['date', 'login_count']], on='date', how='left')
my_train = my_train.fillna(login_size.mean())
train = my_train
train

,date,사용자,세션,신규방문자,페이지뷰,login_count
0,2018-09-09,281,266,73,1826,184.404703
1,2018-09-10,264,247,51,2092,184.404703
2,2018-09-11,329,310,58,1998,184.404703
3,2018-09-12,300,287,45,2595,184.404703
4,2018-09-13,378,344,50,3845,184.404703
...,...,...,...,...,...,...
787,2020-11-04,4516,4472,1196,112683,460.000000
788,2020-11-05,4155,4037,1044,102901,378.000000
789,2020-11-06,3663,3576,825,88015,367.000000
790,2020-11-07,2472,2417,531,57386,224.000000


## 6. 데이터 수정

In [ ]:
# minmax scaling
mini = train.iloc[:,1:-1].min()
size = train.iloc[:,1:-1].max() - train.iloc[:,1:-1].min()
train.iloc[:,1:-1] = (train.iloc[:,1:-1] -  mini) / size

input_window = 30 # 30일씩 데이터를 넣음
output_window = 7 # 이후 7일을 예측

# (a, b, c) -> a: 데이터 수, b: input (이전 30일 데이터, isHoliday), c: 예측 layer
# 사용자 수 -> 
window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window + 1, 4)) # 공휴일여부 추가
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:-1, :] = train.iloc[start : end                , 1: -1].values # 1~30일 사용자수,...4가지
    window_x[start, -1, :] = train.iloc[end, -1]                                   # 31일의 isholiday 
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: -1].values   # 1일~7일 사용자수,....4가지

In [ ]:
window_x.shape, window_y.shape

((755, 31, 4), (755, 7, 4))

## 7. model 생성

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 4, hidden_size = 30).to(device)

## 8. 학습

In [ ]:
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 800
train_error = []
for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 20 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

20 Epochs train MSE: 0.04256
40 Epochs train MSE: 0.02817
60 Epochs train MSE: 0.01801
80 Epochs train MSE: 0.00939
100 Epochs train MSE: 0.00837
120 Epochs train MSE: 0.00820
140 Epochs train MSE: 0.00812
160 Epochs train MSE: 0.00807
180 Epochs train MSE: 0.00805
200 Epochs train MSE: 0.00803
220 Epochs train MSE: 0.00801
240 Epochs train MSE: 0.00800
260 Epochs train MSE: 0.00798
280 Epochs train MSE: 0.00796
300 Epochs train MSE: 0.00795
320 Epochs train MSE: 0.00793
340 Epochs train MSE: 0.00791
360 Epochs train MSE: 0.00789
380 Epochs train MSE: 0.00787
400 Epochs train MSE: 0.00785
420 Epochs train MSE: 0.00782
440 Epochs train MSE: 0.00778
460 Epochs train MSE: 0.00773
480 Epochs train MSE: 0.00767
500 Epochs train MSE: 0.00757
520 Epochs train MSE: 0.00743
540 Epochs train MSE: 0.00720
560 Epochs train MSE: 0.00711
580 Epochs train MSE: 0.00693
600 Epochs train MSE: 0.00674
620 Epochs train MSE: 0.00647
640 Epochs train MSE: 0.00601
660 Epochs train MSE: 0.00559
680 Epochs tra

## 9. 예측하기

In [ ]:
submission = pd.read_csv("/content/gdrive/MyDrive/open_data/submission.csv", encoding = 'euc-kr')
sub_copy = submission.copy()
# sub_copy['date'] = pd.to_datetime(sub_copy['DateTime'])

# sub_copy = utils.add_isHoliday_column(sub_copy)
sub_copy['login_count'] = 0


# sub_holiday_lst = sub_copy.isHoliday.tolist()
sub_login_lst = sub_copy.login_count.tolist()

In [ ]:
sub_copy

,DateTime,사용자,세션,신규방문자,페이지뷰,login_count
0,2020-11-09,0,0,0,0,0
1,2020-11-10,0,0,0,0,0
2,2020-11-11,0,0,0,0,0
3,2020-11-12,0,0,0,0,0
4,2020-11-13,0,0,0,0,0
...,...,...,...,...,...,...
56,2021-01-04,0,0,0,0,0
57,2021-01-05,0,0,0,0,0
58,2021-01-06,0,0,0,0,0
59,2021-01-07,0,0,0,0,0


In [ ]:
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,0,0,0,0
1,2020-11-10,0,0,0,0
2,2020-11-11,0,0,0,0
3,2020-11-12,0,0,0,0
4,2020-11-13,0,0,0,0
...,...,...,...,...,...
56,2021-01-04,0,0,0,0
57,2021-01-05,0,0,0,0
58,2021-01-06,0,0,0,0
59,2021-01-07,0,0,0,0


In [ ]:
last_month

tensor([[[ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.0522, -0.0501, -0.0774, -0.0198],
         [ 0.

In [ ]:
#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = np.zeros((1, 31, 4))
last_month[:, :-1, :] = train.iloc[-30:,1:-1].values[np.newaxis,...] # <- 수정 후
last_month[:, -1, :] = sub_copy.iloc[0, -1]
last_month = torch.tensor(last_month).float().to(device) # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2): # 0, ..., 8
    start = start * 7
    next_week = model(last_month)
    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:-1,:], next_week], axis = 1) # tensor # <- 수정 후

    isholiday_arr = np.array([[[sub_login_lst[start]]*4]])
    isholiday = torch.tensor(isholiday_arr).float().to(device)
    last_month = torch.cat([last_month, isholiday], axis=1)

    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values # scaling 역함수
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2705,2666,666,63299
1,2020-11-10,2972,2890,765,71799
2,2020-11-11,2709,2643,706,65475
3,2020-11-12,2346,2337,594,54895
4,2020-11-13,2009,2046,486,44230
...,...,...,...,...,...
56,2021-01-04,3297,3225,848,82046
57,2021-01-05,2896,2849,748,72028
58,2021-01-06,2389,2404,593,57259
59,2021-01-07,1877,1953,432,41322


## 10. 예측 저장하기

In [ ]:
submission.to_csv('submission_logincount.csv', index = False, encoding = 'euc-kr')